# Loading Data, Packages

In [ ]:
# load spambase dataset


# loading packages


## EDA

In [ ]:
# Exploring distribution of features, target variable in data

## Data Cleaning

In [ ]:
# Normalization, attribute selection via fitting graph

In [ ]:
# other pre-processing considerations (text processing) [try tfidf, if memory issues switch over to G Cloud]

# Modeling: Highest Overall Accuracy
refrence: https://www.researchgate.net/figure/Results-of-Spambase-dataset_tbl2_291833705


In [ ]:
# Decision Tree and Hyperparameter Tuning

In [ ]:
# Knn and Hyperparameter Tuning

In [ ]:
# Naive Bayes and Hyperparameter Tuning

In [ ]:
# LDA and Hyperparameter Tuning

https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28 <br>
"supervised lda", so, for example LDA creates a feature for logistic regression <br>
overall tfidf tends to work better

In [ ]:
# SVM 

In [ ]:
# Feed-Forward Neural Net re: Rodan et. all

In [ ]:
# baseline model performance: AUC graph

In [ ]:
# Model Evaluation: cross-validation, confusion matrix, ROC curve, precision, recall, f-measure 

# Modeling: Best Cost-Sensitive Model
Using 10:1 cost ratio: <br> 
misclassifying a legitimate email as spam is 10 times more costly than classifying spam as a legitimate email

In [ ]:
# Repeat previous models, optimize for cost-sensitivity

In [ ]:
# Model Evaluation: cross-validation, confusion matrix, ROC curve, precision, recall, f-measure

In [ ]:
# Visualization of cost-sensitivity performance

# Discussion of Results
Finally, as a deliverable, produce a write-up (i.e., a single PDF file) describing your 
aforementioned explorations. <br>
Report the performances of different models that you tried (i.e., using different data mining techniques, different attribute selection techniques, etc.) <br> 
What was the performance of the best model in the cost-unaware task (i.e., in terms of accuracy)?<br>
What was the performance of the best model in the cost-aware task (i.e., in 
terms of expected cost)?<br> 
Discuss the best models in two different tasks (as well as their performance) in detail, provide some comparisons. <br>
Draw some conclusions from the assignment.   